In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
from collections import defaultdict
import os

In [2]:
MAX_P = [1]
PROB_1 = [0.95, 0.05]
PROB_2 = [0.95, 0.025, 0.025]
PROB_3 = [0.91, 0.03, 0.03, 0.03]
PROBS = {1: MAX_P, 2: PROB_1, 3: PROB_2, 4: PROB_3}
MCS_PARS_FILE = 'pars_mcs.ini'

In [3]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h, m-h, m+h

In [4]:
# Build dataframes from the simulator's csv log files

def prepare_csv_dfs(directory):
    csv_dfs = defaultdict(dict)
    for entry in tqdm(os.scandir(directory)):
        if os.path.isfile(entry):
            filename = entry.name
            filename = filename[:-4]
            tokens = filename.split('_')
            timestamp = tokens.pop(0)
            timestamp += tokens.pop(0)
            df = pd.read_csv(entry.path)
            csv_dfs[tokens[0] + "_" + timestamp][tokens[2]] = df
            csv_dfs[tokens[0] + "_" + timestamp]['year'] = tokens[1]

    return csv_dfs

In [5]:
# Build a single dataframe including all information from simulator logs

def build_database(dfs):
    index = 0
    count = 0
    rows = []
    for i, race_id in enumerate(tqdm(dfs)):
        tokens = race_id.split('_')
        timestamp = tokens[1]
        race = tokens[0]
        year = dfs[race_id]['year']

        lap_times = dfs[race_id]['laptimes']
        influences = dfs[race_id]['lapinfluences']
        positions = dfs[race_id]['positions']
        race_times = dfs[race_id]['racetimes']
        
        columns = list(lap_times.columns)
        columns.remove('lap')

        for lap in lap_times.lap.unique():
            lap_time_single_lap = lap_times.loc[lap_times['lap'] == lap]
            influences_single_lap = influences.loc[influences['lap'] == lap]
            positions_single_lap = positions.loc[positions['lap'] == lap]
            race_times_single_lap = race_times.loc[race_times['lap'] == lap]
            for col in columns:
                index += 1
                new_df = {}
                new_df['id'] = [timestamp]
                new_df['race'] = [race]
                new_df['year'] = [year]
                new_df['lap'] = [lap]
                new_df['driver'] = [col]
                new_df['lap_time'] = lap_time_single_lap[col].values
                new_df['influence'] = influences_single_lap[col].values
                new_df['cumulative_time'] = race_times_single_lap[col].values
                new_df['position'] = positions_single_lap[col].values
                new_df = pd.DataFrame(new_df, index=[index])
                rows.append(new_df)
    database = pd.concat(rows)
    return database

In [6]:
# Extract strategy statistics
def get_strategy_stats(driver, database, save_directory, log_file_name):
    driver_laps = database.loc[(database['driver'] == driver)]
    driver_laps = driver_laps.sort_values('lap')
    pits = []
    pit_counts = []

    for identifier in driver_laps['id'].unique():
        pit_count = 0
        race_pits = []
        
        unique_laps = driver_laps[driver_laps['id'] == identifier]
        
        for index, row in unique_laps.iterrows():
            if 'pitinlap' in row['influence']:
                pit_count += 1
                race_pits.append((row['lap'], row['influence']))

        pit_counts.append(pit_count)
        pits.append(race_pits)

    with open(os.path.join(save_directory, log_file_name), 'a+') as logfile:

        # print("Average pit count:", np.mean(pit_counts))
        logfile.write("Average pit count: {}".format(np.mean(pit_counts)))
        logfile.write("\n")
        logfile.close()

    plt.figure()
    plt.title("Pit count")
    sns.histplot(pit_counts, discrete=True, shrink=0.8, binrange=(0,5))
    plt.savefig(os.path.join(save_directory, "pit_count.png"))
    # plt.show()


    plt.figure()
    plt.title("Lap time [s]")
    sns.lineplot(data=driver_laps, y='lap_time', x='lap')
    plt.savefig(os.path.join(save_directory, "lap_time.png"))
    # plt.show()


In [7]:
# Extract final placement statistics
def get_final_placement(driver, database, save_directory, log_file_name):
    with open(os.path.join(save_directory, log_file_name), 'a+') as logfile:
        driver_laps = database.loc[(database['driver'] == driver)]
        last_lap = driver_laps['lap'].max()
        last_laps = database[(database['lap'] == last_lap) & (database['driver'] == driver)]

        avg = ("Average final placement: {}".format(last_laps['position'].mean()))
        max_p = ("Max final placement: {}".format(last_laps['position'].min()))
        min_p = ("Min final placement: {}".format(last_laps['position'].max()))
        # print(avg)
        # print(max_p)
        # print(min_p)
        logfile.write(avg + '\n')
        logfile.write(max_p + '\n')
        logfile.write(min_p + '\n')
        logfile.write('\n')
        
        logfile.close()

    last_laps['position'] = last_laps['position'].astype(int)
    plt.figure()
    sns.histplot(data=last_laps, x="position", discrete=True, shrink=0.8, binrange=(1,22))
    plt.title("Final position")
    plt.savefig(os.path.join(save_directory, "final_position.png"))
    # plt.show()

In [8]:
# Simulate true and default strategy

from envs.race_strategy_event import RaceEnv
from race_simulation.racesim.src.import_pars import import_pars

def simulate_strategy(race, driver, strategy_type):

    today = datetime.now()
    timestamp = today.strftime('%Y-%m-%d_%H-%M')
    
    true_policy = False
    quantile_strategies = False
    default_policy = False

    if strategy_type == "true":
        true_policy = True
    elif strategy_type == "quantile":
        quantile_strategies = True
    else:
        default_policy = True
    
    env = RaceEnv(horizon=100, scale_reward=False, start_lap=8, config_path='./envs/configs/race_strategy_event_env_config_notebook.json')
    env.race_config = "pars_" + race + ".ini"
    env.reset()

    csv_save_prefix = race + '/' + strategy_type + '/' + timestamp

    if true_policy:
        pars_in, vse_paths = import_pars(use_print=False,
                                         use_vse=False,
                                         race_pars_file=env.race_config,
                                         mcs_pars_file=MCS_PARS_FILE)

        strategy = defaultdict(int)
        
        true_strategy = pars_in['driver_pars'][driver]['strategy_info']

        if len(true_strategy) == 1:
            print("[WARNING] No pit stop in true strategy!")
        else:
            for pit in true_strategy[1:]:
                strategy[pit[0]] = env.map_compound_to_action(pit[1])


    for i in range(1):
        rews = []

        for _ in trange(100):
            env.reset(quantile_strategies=quantile_strategies)
            lap = 8

            done = False
            cumulative = np.zeros(env.agents_number)
            while not done:
                agent = env.get_next_agent()
                if default_policy or quantile_strategies:
                    actions = env.get_default_strategy(agent)
                    # actions = env.get_available_actions(agent)
                    prob = PROBS[len(actions)]
                    action = np.random.choice(actions, p=prob)
                    # if action > 0:
                    #     print(lap, env.map_action_to_compound(action))
                elif true_policy:
                    action = strategy[lap]

                s, r, done, _ = env.partial_step(action, agent)

                cumulative += r
                lap += 1
                if done:
                    rews.append(cumulative.tolist())
                
            env.save_results(csv_save_prefix)
        # print(rews)
        # print("Return:", np.mean(rews, axis=0))
        # print("std: +-", np.std(rews, axis=0))
        rews = np.array(rews)

    policy_type = "default" if default_policy or quantile_strategies else "human"
    numpy_save_location = "data/RaceStrategy-v2/{}_policy/results_unnorm_{}.npy".format(policy_type, race)
    np.save(numpy_save_location, rews)
    return numpy_save_location, csv_save_prefix


In [9]:
def extract_numpy_scores(path, log_file_name):
    # Extract already stored numpy data
    if not(".npy" in path):
        p = os.path.dirname(path)
        p = os.path.dirname(p)
        root_folder = os.path.dirname(p)
        online_scores = np.load(os.path.join(root_folder, "numpy_dumps", "results.npy"))
    else:
        online_scores = np.load(path)
    mean_ci = mean_confidence_interval(online_scores)
    result_string = "Mean and ci: {} +- {}".format(mean_ci[0].squeeze(), mean_ci[1].squeeze())
    # print(result_string)
    with open(log_file_name, 'a+') as logfile:
        logfile.write(result_string)
        logfile.write("\n")
    return result_string

In [10]:
def pipeline(log, driver):
    
    log_root =os.path.dirname(os.path.dirname(os.path.dirname(log)))

    csv_data = prepare_csv_dfs(log)
    database = build_database(csv_data)
    race = database['race'].unique()[0]
    year = database['year'].unique()[0]
    race_string = str(race) + '_' + str(year)
    # print("### Race:", race, year, "###")
    # print()

    post_string = "postprocessing_" + race_string

    for alg in ["planner", "true", "default"]:
        os.makedirs(os.path.join(log_root, post_string, alg), exist_ok=True)

    log_name = race_string + ".txt"


    # print("Planner")

    save_path = os.path.join(log_root, post_string, "planner")

    extract_numpy_scores(log, os.path.join(save_path, log_name))
    get_strategy_stats(driver, database, save_path, log_name)
    get_final_placement(driver, database, save_path, log_name)
    # print()


    # print("True strategy")

    save_path = os.path.join(log_root, post_string, "true")

    numpy_logs, csv_prefix = simulate_strategy(race_string, driver, "true")
    csv_data = prepare_csv_dfs("./sim_logs/results/" + csv_prefix)
    database = build_database(csv_data)

    extract_numpy_scores(numpy_logs, os.path.join(save_path, log_name))
    get_strategy_stats(driver, database, save_path, log_name)
    get_final_placement(driver, database, save_path, log_name)
    # print()


    # print("Default_strategy")

    save_path = os.path.join(log_root, post_string, "default")

    numpy_logs, csv_prefix = simulate_strategy(race_string, driver, "default")
    csv_data = prepare_csv_dfs("./sim_logs/results/" + csv_prefix)
    database = build_database(csv_data)

    extract_numpy_scores(numpy_logs, os.path.join(save_path, log_name))
    get_strategy_stats(driver, database, save_path, log_name)
    get_final_placement(driver, database, save_path, log_name)
    # print()

In [11]:
import multiprocessing
from multiprocessing.pool import Pool

logs_root = "/mnt/c/Users/Diego/Desktop/Logs/QL_LR_Schedule_a_05_b_2/setting_1"

#logs_list = ["/mnt/c/Users/Diego/Desktop/Logs/Austria_2017_QL_LR_Schedule_a-02_b-2_true_search/sim_logs/results/2020-12-28_15-42-39"]#"/mnt/c/Users/Diego/Desktop/Logs/China_2018_QL_LR_schedule_a05_b2_SC/sim_logs/results/2020-12-24_10-57-51"] #, "/mnt/c/Users/Diego/Desktop/Logs/China_extra_pit/sim_logs/results/2020-12-24_12-02-08"]

driver = "VET"

logs_list = os.scandir(logs_root)

params = []

for dir in logs_list:
    logs_parent = os.path.join(logs_root, dir.name, "sim_logs", "results")
    for experiment in os.scandir(logs_parent):
         params.append((os.path.join(logs_parent, experiment.name), driver))

p = Pool(min(multiprocessing.cpu_count(), len(params)))
p.starmap(pipeline, params) 
p.close()

path = "/mnt/c/Users/Diego/Desktop/Logs/QL_LR_schedule_a_05_b_2/setting_0/SaoPaulo/2021-01-07_11-23-47/numpy_dumps"

file_path = os.path.join(path, "results_80.npy")
online_scores_80 = np.load(file_path).tolist()
file_path = os.path.join(path, "results_20.npy")
online_scores_20 = np.load(file_path).tolist()
online_scores_20.extend(online_scores_80)

final_scores = np.array(online_scores_20)
np.save(os.path.join(path, "results.npy"), final_scores)

rows = []

for parent in os.scandir(logs_root):
    for child in os.scandir(parent):
        if 'postprocessing' in child.name:
            child_root = os.path.join(logs_root, parent.name, child.name)
            row = {}
            for strategy_type in ['true', 'default', 'planner']:
                folder_name = os.path.join(child_root, strategy_type)
                for log_file in os.scandir(folder_name):
                    if 'json' in log_file.name:
                        with open(os.path.join(folder_name, log_file.name), 'r'):
                            row['']



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.DataFrame()

def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in mpl_table._cells.items():
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax.get_figure(), ax

fig,ax = render_mpl_table(df, header_columns=0, col_width=2.0)
fig.savefig("table_mpl.png")